In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
def get_title(soup):

    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            
            price = soup.find("span", attrs={'id':'a-span12'}).string.strip()

        except:
            price = ""

    return price

def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available
if __name__ == '__main__':

    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    URL = "https://www.amazon.com/s?k=Indian+Flag&crid=2Z4O4OM17BU8S&sprefix=indian+flag%2Caps%2C305&ref=nb_sb_noss_1"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
   
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [4]:
amazon_df

,title,price,rating,reviews,availability
0,Double Sided India Flag 3x5ft Heavy Duty Delux...,$19.99,4.5 out of 5 stars,103 ratings,
1,G128 India Indian Flag | 3x5 Ft | LiteWeave Pr...,$12.99,4.5 out of 5 stars,105 ratings,In Stock
2,"Premium USA Indian Flag 3X5Ft Large New, Both ...",$9.99,4.7 out of 5 stars,210 ratings,Only 13 left in stock - order soon.
3,G128 Indiana State Flag | 4x6 Ft | ToughWeave ...,$27.95,4.6 out of 5 stars,270 ratings,In Stock
4,Anley Fly Breeze 3x5 Foot India Flag - Vivid C...,$6.95,4.7 out of 5 stars,"1,383 ratings",In Stock
...,...,...,...,...,...
59,2 Pack India flag Indian Table Flag Small Mini...,$13.99,4.8 out of 5 stars,645 ratings,In Stock
60,"Native American Indian Healing flag 3x5FT, Hea...",$10.99,4.8 out of 5 stars,6 ratings,Only 2 left in stock - order soon.
61,AZ FLAG - USA Indian Chief Flag - 3x5 Ft - 100...,$12.95,4.3 out of 5 stars,110 ratings,Only 8 left in stock - order soon.
62,"Premium USA Indian Flag 3X5Ft Large New, Both ...",$9.99,4.7 out of 5 stars,210 ratings,Only 13 left in stock - order soon.
